In [210]:
import requests
import cloudscraper
import csv
from bs4 import BeautifulSoup
import time

In [211]:
scraper = cloudscraper.create_scraper()
responses = []

In [212]:
for page in range(1, 50 + 1):
    response = scraper.get(f"https://turbo.az/autos?page={page}")
    responses.append(response)
    time.sleep(5)

In [213]:
soups = []
for response in responses:
    soup = BeautifulSoup(response.text, 'html.parser')
    soups.append(soup)

In [214]:
with open("data/turbo-az-cars.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    
    # Write the header
    writer.writerow(["Car Name", "Attributes", "Price", "Currency"])

    # Iterate through each soup
    for soup in soups:
        for product_div in soup.find_all('div', class_='products-i vipped featured'):
            item_div = product_div.find('div', class_='products-i__bottom')
            if item_div:
                price_div = item_div.find('div', class_='products-i__price products-i__bottom-text')
                car_name = item_div.find('div', class_='products-i__name').text.strip()
                car_attrs = item_div.find('div', class_='products-i__attributes').text.strip()
                
                if price_div:
                    full_text = price_div.get_text(strip=True)
                    currency_tag = price_div.find('span')
                    currency = currency_tag.get_text(strip=True) if currency_tag else ''
                    price = full_text.replace(currency, '').strip()
                else:
                    price = ''
                    currency = ''

                # Write the row to the CSV
                writer.writerow([car_name, car_attrs, price, currency])
